In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False

def rk4(f, t0, y0, t1, dt, params):
    """Simple fixed-step RK4 integrator.
    y0: 1D numpy array
    returns ts, ys
    """
    n = int(math.ceil((t1 - t0) / dt)) + 1
    ts = np.linspace(t0, t0 + dt*(n-1), n)
    ys = np.zeros((n, len(y0)), dtype=float)
    ys[0] = y0.copy()

    y = y0.copy()
    t = t0
    for i in range(1, n):
        k1 = f(t, y, params)
        k2 = f(t + dt/2, y + dt*k1/2, params)
        k3 = f(t + dt/2, y + dt*k2/2, params)
        k4 = f(t + dt,   y + dt*k3,   params)
        y = y + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)
        t = ts[i]

        # Simple state constraints: project K, L to be nonnegative
        y = np.maximum(y, 0.0)

        ys[i] = y

    return ts, ys


In [ ]:
def model(t, y, p):
    # State variables:
    # K: capital stock
    # L: employed labour (or a proxy for activity)
    # w: wage share proxy (bounded in [0, 1] by projection later if you want)
    K, L, w = y

    # Parameters
    a  = p["a"]   # labour productivity (output per labour)
    s  = p["s"]   # propensity to save/invest out of profits
    d  = p["d"]   # depreciation
    n  = p["n"]   # labour force growth / demand growth
    phi = p["phi"] # wage adjustment speed
    w_star = p["w_star"] # target wage share

    # Flows
    Y = a * L
    # simple profit share = (1 - w)
    P = (1 - w) * Y

    # Accumulation: dK/dt = s*P - d*K
    dK = s * P - d * K

    # Activity / employment dynamics: stylised, respond to accumulation and exogenous growth
    dL = n * L + 0.05 * (dK / (K + 1e-9)) * L

    # Wage share adjustment: move towards w_star, but push up when labour is “tight”
    tightness = L / (L + K + 1e-9)
    dw = phi * (w_star - w) + 0.2 * phi * (tightness - 0.5)

    return np.array([dK, dL, dw], dtype=float)


In [ ]:
def run_sim(params, t1=200.0, dt=0.1, y0=(50.0, 10.0, 0.6)):
    ts, ys = rk4(model, 0.0, np.array(y0, dtype=float), t1, dt, params)
    K = ys[:,0]
    L = ys[:,1]
    w = ys[:,2]
    Y = params["a"] * L
    profit_rate = ((1 - w) * Y) / (K + 1e-9)

    plt.figure()
    plt.plot(ts, K, label="K (capital)")
    plt.plot(ts, L, label="L (activity)")
    plt.plot(ts, w, label="w (wage share proxy)")
    plt.legend()
    plt.xlabel("t")
    plt.show()

    plt.figure()
    plt.plot(ts, profit_rate, label="profit rate")
    plt.legend()
    plt.xlabel("t")
    plt.show()

default_params = dict(a=1.0, s=0.25, d=0.03, n=0.01, phi=0.05, w_star=0.6)
run_sim(default_params)


In [ ]:
if HAS_WIDGETS:
    display(Markdown("### Foley playground controls"))

    a = widgets.FloatSlider(value=1.0, min=0.2, max=3.0, step=0.1, description="a")
    s = widgets.FloatSlider(value=0.25, min=0.0, max=0.9, step=0.01, description="s")
    d = widgets.FloatSlider(value=0.03, min=0.0, max=0.2, step=0.005, description="d")
    n = widgets.FloatSlider(value=0.01, min=-0.02, max=0.05, step=0.001, description="n")
    phi = widgets.FloatSlider(value=0.05, min=0.0, max=0.5, step=0.01, description="phi")
    w_star = widgets.FloatSlider(value=0.6, min=0.1, max=0.9, step=0.01, description="w*")
    t1 = widgets.FloatSlider(value=200, min=20, max=400, step=10, description="T")
    dt = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description="dt")

    ui = widgets.VBox([a, s, d, n, phi, w_star, t1, dt])
    out = widgets.Output()

    def on_change(change=None):
        params = dict(a=a.value, s=s.value, d=d.value, n=n.value, phi=phi.value, w_star=w_star.value)
        with out:
            out.clear_output(wait=True)
            run_sim(params, t1=float(t1.value), dt=float(dt.value))

    for wdg in [a, s, d, n, phi, w_star, t1, dt]:
        wdg.observe(on_change, names="value")

    display(ui, out)
    on_change()
else:
    print("Widgets not available in this environment; run_sim(default_params) still works.")
